In [1]:
import pysal as ps
import numpy as np
import json
from collection import Point, MultiPoint, LineString, MultiLineString
from collection import Polygon, MultiPolygon
from collection import geometryDispatcher, FeatureCollection

In [2]:
ps.examples.explain('virginia')

{'description': 'Virginia counties shapefile',
 'explanation': ['  * virginia.shp: Shapefile',
  '  * virginia.shx: shapefile index',
  '  * virginia.dbf: attributes',
  '  * virginia.prj: shapefile projection',
  '  * virginia.json: geojson file'],
 'name': 'virginia'}

In [3]:
fp = ps.examples.get_path('virginia.json')
vfc = FeatureCollection(fp)

In [4]:
vfc.getPropertyTypes()

{u'CNTY_FIPS': unicode,
 u'FIPS': unicode,
 u'Key': int,
 u'NAME': unicode,
 u'POLY_ID': int,
 u'STATE_FIPS': unicode,
 u'STATE_NAME': unicode}

In [5]:
f0 = vfc.features[0]
f0['properties']

{u'CNTY_FIPS': u'069',
 u'FIPS': u'51069',
 u'Key': 1147,
 u'NAME': u'Frederick',
 u'POLY_ID': 1,
 u'STATE_FIPS': u'51',
 u'STATE_NAME': u'Virginia'}

In [6]:
print(vfc.features[0]['geometry'])

MultiPolygon: 1 Polygon(s)


In [7]:
for feature in vfc.features:
    print(vfc.features[feature]['geometry'])

MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
MultiPolygon: 1 Polygon(s)
M

In [8]:
fp = ps.examples.get_path('columbus.json')
cfc = FeatureCollection(fp)

In [9]:
print(cfc.features[0]['geometry'])

Polygon: 1 Rings, 0 Holes


In [10]:
cfc.getPropertyTypes()

{u'AREA': float,
 u'COLUMBUS_': float,
 u'COLUMBUS_I': float,
 u'CP': float,
 u'CRIME': float,
 u'DISCBD': float,
 u'EW': float,
 u'HOVAL': float,
 u'INC': float,
 u'NEIG': int,
 u'NEIGNO': float,
 u'NSA': float,
 u'NSB': float,
 u'OPEN': float,
 u'PERIMETER': float,
 u'PLUMB': float,
 u'POLYID': float,
 u'THOUS': float,
 u'X': float,
 u'Y': float}

In [11]:
X = cfc.getPropertiesAsArray(['AREA', 'CRIME'])

In [12]:
XL = cfc.getPropertiesAsLists(['AREA', 'CRIME'])
XL

[[0.309441, 15.72598],
 [0.259329, 18.801754],
 [0.192468, 30.626781],
 [0.083841, 32.38776],
 [0.488888, 50.73151],
 [0.283079, 26.066658],
 [0.257084, 0.178269],
 [0.204954, 38.425858],
 [0.500755, 30.515917],
 [0.246689, 34.000835],
 [0.041012, 62.275448],
 [0.035769, 56.705669],
 [0.034377, 46.716129],
 [0.060884, 57.066132],
 [0.106653, 48.585487],
 [0.093154, 54.838711],
 [0.102087, 36.868774],
 [0.055494, 43.962486],
 [0.061342, 54.521965],
 [0.444629, 0.223797],
 [0.699258, 40.074074],
 [0.192891, 33.705048],
 [0.24712, 20.048504],
 [0.192226, 38.297871],
 [0.17168, 61.299175],
 [0.107298, 40.969742],
 [0.137802, 52.79443],
 [0.174773, 56.919785],
 [0.085972, 60.750446],
 [0.104355, 68.892044],
 [0.117409, 17.677214],
 [0.18558, 19.145592],
 [0.087472, 41.968163],
 [0.226594, 23.974028],
 [0.175453, 39.175053],
 [0.17813, 14.305556],
 [0.121154, 42.445076],
 [0.053881, 53.710938],
 [0.174823, 19.100863],
 [0.302908, 16.241299],
 [0.137024, 18.905146],
 [0.266541, 16.49189],
 [0

In [13]:
g = cfc.getGeometryCollection()

In [14]:
type(g)

generator

In [15]:
cfc.features[0]['geometry']

In [16]:
dir(cfc.features[0]['geometry'])

['_Geometry__bbox',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bbox',
 'bboxes',
 'build_bbox',
 'coordinates',
 'getbbox',
 'setbbox',
 'type']

In [17]:
cfc.features[0]['geometry'].coordinates

[[[8.624129295349121, 14.236980438232422],
  [8.559700012207031, 14.742449760437012],
  [8.809452056884766, 14.734430313110352],
  [8.808412551879883, 14.636520385742188],
  [8.919304847717285, 14.638500213623047],
  [9.087138175964355, 14.63049030303955],
  [9.09996509552002, 14.244830131530762],
  [9.015047073364258, 14.241840362548828],
  [9.008951187133789, 13.995059967041016],
  [8.818140029907227, 14.002050399780273],
  [8.653305053710938, 14.008090019226074],
  [8.642902374267578, 14.089710235595703],
  [8.63259220123291, 14.1705904006958],
  [8.625825881958008, 14.22367000579834],
  [8.624129295349121, 14.236980438232422]]]

In [18]:
def geojson_handler(file_path):
    with open(file_path) as source:
        data = json.load(source)
    feature_type = data['features'][0]['geometry']['type']
    return data

In [19]:
data = geojson_handler(ps.examples.get_path('virginia.json'))

In [20]:
data.keys()

[u'crs', u'type', u'features']

In [21]:
f0

{'geometry': <collection.MultiPolygon object at 0x7f07686b0dd0>,
 'properties': {u'CNTY_FIPS': u'069',
  u'FIPS': u'51069',
  u'Key': 1147,
  u'NAME': u'Frederick',
  u'POLY_ID': 1,
  u'STATE_FIPS': u'51',
  u'STATE_NAME': u'Virginia'}}

In [22]:
f0

{'geometry': <collection.MultiPolygon object at 0x7f07686b0dd0>,
 'properties': {u'CNTY_FIPS': u'069',
  u'FIPS': u'51069',
  u'Key': 1147,
  u'NAME': u'Frederick',
  u'POLY_ID': 1,
  u'STATE_FIPS': u'51',
  u'STATE_NAME': u'Virginia'}}

In [23]:
c =  [
      [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0] ],
      [ [100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2] ]
      ]
p = Polygon(c)
p.coordinates
print(p.bbox)
p.bboxes

c = [
      [[[102.0, 2.0], [103.0, 2.0], [103.0, 3.0], [102.0, 3.0], [102.0, 2.0]]],
      [[[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]],
       [[100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2]]]
      ]

print("\n")
mp = MultiPolygon(c)

print(mp.bbox)

c =  [ [100.0, 0.0], [101.0, 1.0] ]
l = LineString(c)
print(l.bbox)

c = [
        [ [100.0, 0.0], [101.0, 1.0] ],
        [ [102.0, 2.0], [103.0, 3.0] ]
      ]

mls = MultiLineString(c)
mls.bbox

c = [100.0, 0.0]
p = Point(c)
p.bbox

c = [ [100.0, 0.0], [101.0, 1.0] ]

np.array(c).max(axis=0)
mp = MultiPoint(c)
mp.bbox

c = [ [100.0, 0.0], [101.0, 1.0], [40, 7]]
MultiPoint(c).bbox

c = [ [100.0, 0.0, 20], [101.0, 1.0, 50], [40, 7, 10]]
MultiPoint(c).bbox

str(mp)

[100.0, 0.0, 101.0, 1.0]


[100.0, 0.0, 103.0, 3.0]
(100.0, 0.0, 101.0, 1.0)


'MultiPoint: [[100.0, 0.0], [101.0, 1.0]]'

In [24]:
str(mp)

'MultiPoint: [[100.0, 0.0], [101.0, 1.0]]'

In [25]:
geometryDispatcher = {}
geometryDispatcher['Point'] = Point

In [28]:
v.type

'MultiPolygon'

In [29]:
b.type

'Point'

In [30]:
b.geometries

<generator object <genexpr> at 0x7f0768588b40>

In [31]:
c.type

'Polygon'

In [32]:
for g in c.geometries:
    print g

In [33]:
c.bbox

[5.87490701675415, 11.287420272827148, 10.788629531860352, 14.742449760437012]

In [34]:
b.bbox

[255.0, 247.0, 335.0, 399.0]

In [39]:
def bbox_overlap(A, B):
    """
    Test if two bounding boxes overlap
    
    Arguments
    ---------
    A, B: list of [left, bottom, right, top]
    """
    l = 0
    b = 1
    r = 2
    t = 3
    if A[b] > B[t] or A[t] < B[b] or A[r] < B[l] or A[l] > B[r]:
        return False
    else:
        return True
    
b1 = [0,0, 10,10]
b2 = [11,0, 10, 10]
bbox_overlap(b1, b2)
b2 = [5,5, 10, 10]
bbox_overlap(b1, b2)
b2 = [-5,5, -1, 10]
bbox_overlap(b1, b2)
b2 = [10,10, 20,20]
bbox_overlap(b1, b2)



True

In [40]:
cfc

In [221]:
def contiguous(g1, g2, criterion='QUEEN'):
    if not bbox_overlap(g1.bbox, g2.bbox):
        return False
    else:
        #print('checking geometries')
        if g1.type == g2.type:
            #print('same types')
            gtype = g1.type, g2.type
            overlap = False
            if g1.type == 'Polygon':
                pairs = []
                for i, bbox1 in enumerate(g1.bboxes):
                    for j, bbox2 in enumerate(g2.bboxes):
                        if bbox_overlap(bbox1, bbox2):
                            overlap=True
                            pairs.append((i,j))
                            #break
                #print('still in if broke out of 2fl')
                #print pairs
            elif g1.type == 'MultiPolygon':
                #print 'mp'
                pairs = []
                for i, pi in enumerate(g1.bboxes):
                    for ii, bi in enumerate(pi):
                        for j, pj in enumerate(g2.bboxes):
                            for jj, bj in enumerate(pj):
                                if bbox_overlap(bi, bj):
                                    overlap = True
                                    pair = (i,ii,j,jj)
                                    pairs.append(pair)
                                    #break
                #print(pairs)
                #print('still in if broke out of 4fl')
                if overlap:
                    #print('explict check to be done')
                    if criterion.lower()== 'queen':
                        queen = False
                        for pair in pairs:
                            #print(pair)
                            i, ii, j, jj = pair
                            pi = g1.bboxes[i][ii]
                            pj = g2.bboxes[j][jj]
                            npi = len(pi)
                            npj = len(pj)
                            cis = [pi[l:l+2] for l in range(npi-1)]
                            cjs = [pj[l:l+2] for l in range(npj-1)]
                            iinj = [ci for ci in cis if ci in cjs]
                            if iinj:
                                queen = True
                                break
                        return queen
                    else: # rook
                        rook = False
                        for pair in pairs:
                            #print(pair)
                            i, ii, j, jj = pair
                            pi = g1.bboxes[i][ii]
                            pj = g2.bboxes[j][jj]
                            npi = len(pi)
                            npj = len(pj)
                            cis = [pi[l:l+2] for l in range(npi-1)]
                            cjs = [pj[l:l+2] for l in range(npj-1)]
                            iinj = [ci for ci in cis if ci in cjs]
                            if iinj:
                                ncis = len(cis)
                                ncjs = len(cjs)
                                eis = [sorted(cis[l:l+2]) for l in range(ncis-1)]
                                ejs = [sorted(cjs[l:l+2]) for l in range(ncjs-1)]
                                eiinj = [ei for ei in eis if ei in ejs]
                                if eiinj:
                                    rook = True
                                    #print 'rook neighbors'
                                    break
                        return rook
                else:
                    return False
                
        else:
            return False

       

In [222]:
g41 = vfc.features[41]['geometry']
g41c = vfc.features[41]['geometry']
g0 = vfc.features[0]['geometry']
print(contiguous(g41, g41c))
print(contiguous(g0, g41))

True
False


In [224]:
print(contiguous(g41, g41c, 'rook'))

True


In [225]:
vfc.n_features

136

In [ ]:
neighbors = dict([(i,[]) for i in xrange(vfc.n_features)])
for i in xrange(vfc.n_features - 1):
    for j in xrange(i+1, vfc.n_features):